In [206]:
import csv
from glob import glob
import pandas as pd
import os
from datetime import datetime

total_list = glob(r"Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\*")
data_list = []
for i in total_list: # i => Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_0413
    id_list = glob(r"{}\*".format(i))
    u_id = i.split("\\")[-1]
    t1_list = []
    
    # 같은 id 내에 하위 폴더들을 돌면서 리스트에 t1 데이터 추가(경로, 시간)
    for j in id_list: # j => ADNI_MRI_PET_pair\002_S_0413\002_S_0413_20170615
        id_mri_list = glob(r"{}\*".format(j))
        for k in id_mri_list:
            if "t1" in k.split("\\")[-1]:
                t1_dict = {"t1_path": k, "t1_date": k.split("\\")[-2].split("_")[-1]}
                t1_list.append(t1_dict)

    for l in id_list:  # l => ADNI_MRI_PET_pair\002_S_0413\002_S_0413_20170615
        id_mri_list = glob(r"{}\*".format(l))
        # id_mri_list에 pet 폴더가 없는 경우 다음 루프로 넘어감
        pet_check = list(map(lambda x: "pet" in x.split("\\")[-1], id_mri_list))
        if pet_check.count(True) == 0:
            continue
        else:
            t1_path = None
            t1_date = None
            pet_path = id_mri_list[pet_check.index(True)]
            # 파일 이름 형식이 다른 경우가 1가지만 있어서 if문으로 처리함
            if pet_path == r"Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\003_S_6264\2018-09-24_15_27_20.0\pet_amyloid":
                pet_date = "20180924"
            else:
                pet_date = pet_path.split("\\")[-2].split("_")[-1]
                
            # mri 폴더 중에 t1 폴더가 있으면 t1_path와 t1_date 값 할당
            for m in id_mri_list:
                if "t1" in m:
                    t1_path = m
                    t1_date = m.split("\\")[-2].split("_")[-1]
            # 반복문 수행 후 t1Path와 t1_path 값이 None이면 같은 id 내의 다른 폴더에서 가져옴
            if t1_path is None:
                # 다른 폴더 내에도 t1 폴더가 없을 경우
                if len(t1_list) == 0:
                    t1_path = "No t1"
                    t1_date = "No t1"
                    continue
                # pet_date와의 차이를 계산
                time_check = list(map(lambda x: abs(datetime.strptime(pet_date, '%Y%m%d') - datetime.strptime(x["t1_date"], '%Y%m%d')), t1_list))
                # 가장 차이가 적은 t1 폴더의 경로와 시간 할당
                index = time_check.index(min(time_check))
                t1_path = t1_list[index]["t1_path"]
                t1_date = t1_list[index]["t1_date"]
            # pet_date - t1_date 
            gap = datetime.strptime(pet_date, '%Y%m%d') - datetime.strptime(t1_date, '%Y%m%d')

            data_list.append([u_id, t1_path, pet_path, t1_date, pet_date, gap])
            
            if pet_path == r"Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\003_S_6264\2018-09-24_15_27_20.0\pet_amyloid":
                print([u_id, t1_path, pet_path, t1_date, pet_date, gap])
                print(data_list)

# list -> dataframe
df = pd.DataFrame(data_list, columns=['id', 't1_path', 'pet_path', 't1_date', 'pet_date', 'gap'])
df
# df.to_csv("result.csv", index=False)

['003_S_6264', 'Z:\\Img_Opensource\\nii\\ADNI_MRI_PET_pair\\003_S_6264\\003_S_6264_20180604\\t1', 'Z:\\Img_Opensource\\nii\\ADNI_MRI_PET_pair\\003_S_6264\\2018-09-24_15_27_20.0\\pet_amyloid', '20180604', '20180924', datetime.timedelta(days=112)]
[['002_S_0413', 'Z:\\Img_Opensource\\nii\\ADNI_MRI_PET_pair\\002_S_0413\\002_S_0413_20170621\\t1', 'Z:\\Img_Opensource\\nii\\ADNI_MRI_PET_pair\\002_S_0413\\002_S_0413_20170615\\pet_amyloid', '20170621', '20170615', datetime.timedelta(days=-6)], ['002_S_0413', 'Z:\\Img_Opensource\\nii\\ADNI_MRI_PET_pair\\002_S_0413\\002_S_0413_20170621\\t1', 'Z:\\Img_Opensource\\nii\\ADNI_MRI_PET_pair\\002_S_0413\\002_S_0413_20170621\\pet_tau', '20170621', '20170621', datetime.timedelta(0)], ['002_S_0413', 'Z:\\Img_Opensource\\nii\\ADNI_MRI_PET_pair\\002_S_0413\\002_S_0413_20190827\\t1', 'Z:\\Img_Opensource\\nii\\ADNI_MRI_PET_pair\\002_S_0413\\002_S_0413_20190813\\pet_amyloid', '20190827', '20190813', datetime.timedelta(days=-14)], ['002_S_1155', 'Z:\\Img_Openso

,id,t1_path,pet_path,t1_date,pet_date,gap
0,002_S_0413,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,20170621,20170615,-6 days
1,002_S_0413,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,20170621,20170621,0 days
2,002_S_0413,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,20190827,20190813,-14 days
3,002_S_1155,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,20170424,20170420,-4 days
4,002_S_1155,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,20170424,20170424,0 days
...,...,...,...,...,...,...
1736,941_S_6581,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\941_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\941_S_...,20180912,20180919,7 days
1737,941_S_6581,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\941_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\941_S_...,20180912,20180921,9 days
1738,941_S_6581,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\941_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\941_S_...,20200925,20201008,13 days
1739,941_S_6803,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\941_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\941_S_...,20190916,20191003,17 days


,id,t1_path,pet_path,t1_date,pet_date,gap
0,002_S_0413,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,20170621,20170615,-6 days
1,002_S_0413,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,20170621,20170621,0 days
2,002_S_0413,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,20190827,20190813,-14 days
3,002_S_1155,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,20170424,20170420,-4 days
4,002_S_1155,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,20170424,20170424,0 days
5,002_S_1155,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,20190516,20190513,-3 days
6,002_S_1261,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,20170315,20170314,-1 days
7,002_S_1261,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,20170315,20170315,0 days
8,002_S_1261,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,20190501,20190318,-44 days
9,002_S_1280,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,Z:\Img_Opensource\nii\ADNI_MRI_PET_pair\002_S_...,20170313,20170302,-11 days
